# Práctica 1. Administración básica de un servidor

Los objetivos de esta práctica son:
- Poner en marcha un servidor linux accesible
- Securizarlo y programar un script de mantenimiento
- Instalar y mantener levantada una api-rest en python

## Paso a paso:

## 1. Administración básica de un servidor:
- `Crear una instancia de EC2 en AWS (Amazon Elastic Compute Cloud) utilizando la consola de AWS.`
- Conectar a la instancia utilizando SSH desde tu terminal o cliente SSH favorito.

__Conexión SSH:__
1. Entrar en _Visual Studio Code_ y conectarse a la máquina virtual
2. Moverse hasta la carpeta en la que se encuentra nuestra clave de entrada al servidor
3. Ejecutar este comando: `chmod 400 "Nombre_de_la_clave.pem"` en terminal cambiando `Nombre_de_la_clave` por el de la nuestra
4. Ir a `conectar` en AWS al seleccionar la instancia y copiar el ejemplo y cambiar el nombre de la llave.<br> Queda algo como esto: `ssh -i "Llave-Mi_servidor_web.pem" ubuntu@ec2-13-53-136-17.eu-north-1.compute.amazonaws.com`

In [ ]:
~$ chmod 400 "Llave-Mi_servidor_web.pem"

## 2. Puesta en marcha de un servidor Linux accesible:
- `Elegir una imagen de sistema operativo Linux compatible, como Amazon Linux, Ubuntu, CentOS, etc., al crear la instancia EC2.`
- Configurar reglas de seguridad en el grupo de seguridad de la instancia EC2 para permitir el acceso a los puertos necesarios (por ejemplo, SSH en el puerto 22 para acceso remoto).

__Cerrar puertos de entrada y salida en desuso:__
> Identificar el ID del Grupo de Seguridad: `curl -s http://169.254.169.254/latest/meta-data/security-groups/`<br>
> Revocar el acceso a todos los puertos excepto el 22: `aws ec2 revoke-security-group-ingress --group-id tu-id-de-grupo --protocol all --cidr 0.0.0.0/0`

Comprovación de que únicamente el puerto 22, de conexión ssh está abierto y vemos que se puede acceder desde cualquier terminal (0.0.0.0/0) 

![alt text](Puertos_de_entrada_1.png)

## 3. Securización y programación de script de mantenimiento:

### Pasos para implementar Fail2ban:

1. Descargar Python (versión mayor o igual a 3.5)
    1. Actualizar el sistema: `sudo apt update`
    2. Instalar python: `sudo apt install python3 python3-pip`
    3. Verificar la instalación: `pip3 --version`
2. Ejecutar comando de instalación: `sudo apt install fail2ban`
3. Configurar _fail2ban_:
    1. Nos movemos al directorio de fail2ban: `cd /etc/fail2ban`
    2. Cambiar de nombre el archivo de fail.conf a fail.local
    3. Modificar el tiempo de baneo: por 3 intentos fallados en 5 min se banea 48h


### Realice un script de shell que:
- Realice copias de seguridad de los principales directorios y ficheros
- Borre los ficheros de log más antiguos de 7 dias
- Se ejecte todas las madrugadas

#### Script:

In [ ]:
#!/bin/bash

# Rutas de directorios a respaldar y directorio de copias de seguridad
directorios_respaldar="/etc /var/lib /home/*"
directorio_backup="/home/directorio_backup"

# Copia de seguridad de directorios
fecha=$(date +"%Y-%m-%d")
tar_file="$directorio_backup/backup_$fecha.tar.gz"
tar -czvf "$tar_file" $directorios_respaldar

# Eliminar archivos de log más antiguos de 7 días
find /ruta/a/logs -type f -name "*.log" -mtime +7 -exec rm {} \;

# Mensaje de registro
echo "Copia de seguridad realizada en $tar_file"
echo "Archivos de log más antiguos de 7 días eliminados."

Explicación del script:
1. directorios_respaldar: Lista de directorios que deseas respaldar.
2. directorio_backup: Directorio donde se almacenarán las copias de seguridad.
3. Se crea una copia de seguridad comprimida (tar.gz) de los directorios especificados en la variable directorios_respaldar.
4. Se eliminan los archivos de log más antiguos de 7 días del directorio /ruta/a/logs.
5. Se registra un mensaje de confirmación en la terminal.

#### Automatización:

1. Ejecutar el siguiente comando para modificar el cron: `crontab -e`
2. Pegar la siguiente línea al final del archivo: `0 3 * * * /home/scripts/script_backup.sh > /home/directorio_backup/log_backup.log 2>&1` <BR> Esta línea ejecuta el script de recuperación todos los días a las 3 am.

## 4. Servicios:

### 1. Crear la API mínima con Bottle:

1. Instalar Bottle: `pip install bottle`

2. Crear un archivo python que implemente los siguintes recursos:
        - /hi : "Hola, hoy es dd/MM/YYYY y son las HH:mm:ss"
        - /status: lista los servicios que están corriendo en la máquina

In [ ]:
from bottle import Bottle, run
import datetime

app = Bottle()

@app.route('/hi')
def hello():
    now = datetime.datetime.now()
    return f'Hola, hoy es {now.strftime("%d/%m/%Y")} y son las {now.strftime("%H:%M:%S")}'

@app.route('/status')
def status():
    process = subprocess.Popen(['systemctl', 'list-units', '--type=service', '--state=running'], stdout=subprocess.PIPE)
    output, error = process.communicate()
    lines = output.decode('utf-8').split('\n')
    services = []
    for line in lines:
        parts = line.split()
        if len(parts) >= 4:
            services.append({
                'unit': parts[0],
                'load': parts[1],
                'active': parts[2],
                'sub': parts[3],
                'description': ' '.join(parts[4:])
            })

    # Genera una tabla HTML
    html = '<table border="1">'
    html += '<tr><th>Unit</th><th>Load</th><th>Active</th><th>Sub</th><th>Description</th></tr>'
    for service in services:
        html += f'<tr><td>{service["unit"]}</td><td>{service["load"]}</td><td>{service["active"]}</td><td>{service["sub"]}</td><td>{service["description"]}</td></tr>'
    html += '</table>'

    response.content_type = 'text/html'
    return html
   

if __name__ == '__main__':
    run(app, host='0.0.0.0', port=8080)

3. Darle permisos de ejecución al archivo: `chmod +x /home/scripts/api.py`
4. Permitir el acceso de tráfico entrante por el puerto 80 ejecutando: `iptables -A INPUT -p tcp --dport 8080 -j ACCEPT`
5. Redirigir el tráfico del puerto 8080 al puerto 80 (donde está la API):<br> `iptables -t nat -A PREROUTING -p tcp --dport 8080 -j REDIRECT --to-port 80`
6. Crear el directorio iptables en /etc/: `sudo mkdir -p /etc/iptables/`
7. Guardar las reglas de iptables: `sudo iptables-save > /etc/iptables/rules.v4`
8. Meterse en `Security Groups` / `launch-wizard-1`
9. Editar las reglas de entrada y **crear** una nueva regla `TCP personalizado` con el puerto `8080` y que pueda acceder cualquier dirección ip `0.0.0.0/0`
10. Repetirlo para las reglas de salida

### 2. Instalar y configurar Supervisor:

1. Ejecutar el siguiente comando: `sudo apt-get install supervisor`
2. Crea un archivo de configuración para tu aplicación en Supervisor. En este caso, api.conf en /etc/supervisor/conf.d:

In [ ]:
[program:api]   # Llamarlo igual que la api
command=/usr/bin/python3 /home/scripts/api.py  # Ruta absoluta al script api.py
directory=/home/scripts  # Directorio donde se encuentra api.py
autostart=true
autorestart=true
stderr_logfile=/var/log/api.err.log
stdout_logfile=/var/log/api.out.log


3.  Actualiza la configuración de Supervisor y comienza a supervisar tu aplicación:<BR>
    `sudo supervisorctl reread`<br>
    `sudo supervisorctl update`<br>
    `sudo supervisorctl start my_api`


Prueba de que la api funciona:
- Prueba del programa hi: `curl http://localhost:8080/hi`
- Prueba del programa status: `curl http://localhost:8080/status`

### 3. Instalar Nginx y configurar un proxy inverso:

1. Instalar Nginx: `sudo apt-get install nginx`
2. Creamos un archivo de configuración para la API en Nginx. En nuestro caso, api.conf en /etc/nginx/sites-available/:

In [ ]:
server {
    listen 80;
    server_name tu_dominio_o_ip;

    location / {
        proxy_pass http://127.0.0.1:8080;  # Puerto donde se ejecuta la API con Bottle
        proxy_set_header Host $host;
        proxy_set_header X-Real-IP $remote_addr;
        proxy_set_header X-Forwarded-For $proxy_add_x_forwarded_for;
    }
}


3. Habilitamos el sitio y reiniciamos Nginx: <br>
    `sudo ln -s /etc/nginx/sites-available/api.conf /etc/nginx/sites-enabled/` <br>
    `sudo systemctl restart nginx`


`Hacer un programa en CRON para poder lanzar el servidor constantemente`

# Práctica 2: Servidor de STOP!

### Crear los programas en python para conectar los sockets desde ambas direcciones:

#### Programa cliente:

In [ ]:
# El programa cliente que se comunica con el Servidor Software que está escuchando en el Servidor Físico
import socket

# El host remoto. Sería "localhost" si el cliente está en la misma máquina que el servidor
HOST = "13.53.94.147"

# El puerto en el que está escuchando nuestro servidor
PORT = 50000


if __name__ == "__main__":
	print('[Cliente 1] Iniciando socket con el host "{}" al puerto {}'.format(HOST, PORT))
	with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as mi_socket:
		tupla_para_el_enlace = (HOST, PORT)

		print('[Cliente 2] Conectando a: {}'.format(tupla_para_el_enlace))
		mi_socket.connect(tupla_para_el_enlace)

		mantener_el_socket_abierto = True
		while mantener_el_socket_abierto:
			dato_para_enviar_str = input("[Cliente 3] Escribe el dato a enviar al servidor: ")

			if dato_para_enviar_str == "cerrar":
				print("[Cliente 6] Cerrando socket desde cliente")
				break

			mi_socket.sendall(bytes(dato_para_enviar_str, encoding='utf8'))

			print('[Cliente 4] Datos enviados, esperando respuesta del servidor ...')
			dato_recibido_en_bytes = mi_socket.recv(1024).strip()

			dato_recibido_en_str = dato_recibido_en_bytes.decode("utf-8")
			print("[Cliente 5] Respuesta recibida desde el servidor: {}".format(dato_recibido_en_str))

		print("[Cliente 7] Socket cerrado desde cliente")

#### Programa servidor:

##### Moverse al siguiente directorio para guardar los scripts:

In [ ]:
cd /usr/local/bin

#### Código programa:

In [ ]:
# Nuestro Servidor Software sobre el Servidor Físico
import socketserver

# Dirección IP
HOST = "0.0.0.0"

# El puerto privado que queramos escuchar, uno de los comprendidos entre 49152 y 65535  1-65535
PORT = 50000


class MiControladorTCP(socketserver.BaseRequestHandler):
    """
    La clase que controlará las peticiones para nuestro servidor.

    It is instantiated once per connection to the server, and must
    override the handle() method to implement communication to the
    client.
    """

    def handle(self):
        """
        Método sobrescrito para controlar la comunicación que ocurra ne nuestro servidor.
        Aquí recibiremos los mensajes del cliente y le responderemos
        """
        print('[Servidor 3] Cliente conectado desde: {}'.format(self.client_address[0]))

        socket_abierto = True
        while socket_abierto:
            print('[Servidor 4] Esperando por petición del cliente...')
            dato_recibido_en_bytes = self.request.recv(1024).strip()
            if dato_recibido_en_bytes:
                dato_recibido_en_str = dato_recibido_en_bytes.decode("utf-8") 
                print('[Servidor 5] Recibido desde el cliente: {}'.format(dato_recibido_en_str))

                respuesta_en_str = "## RESPUESTA DEL SERVIDOR: {} ##".format(dato_recibido_en_str)
                
                self.request.sendall(bytes(respuesta_en_str, encoding='utf8'))
                print('[Servidor 6] Se ha respondido al cliente con el mensaje: {}'.format(respuesta_en_str))
            else:
                print('[Servidor 7] El cliente ha cerrado el Socket desde su lado, cerrando socket desde el Servidor...')
                socket_abierto = False


if __name__ == "__main__":
    tupla_para_el_enlace = (HOST, PORT)

    try:
        print('[Servidor 1] Enlazando Socket en: {}'.format(tupla_para_el_enlace))
        with socketserver.TCPServer(tupla_para_el_enlace, MiControladorTCP) as servidor:
            print('[Servidor 2] Iniciando buble del servidor. Para interrumpir pulsar a la vez las teclas: [Ctrl]+[C]')
            servidor.serve_forever()
    except KeyboardInterrupt:
        print('[Servidor 8] Interrupción por teclado')
    finally:        
        if servidor is not None:
            servidor.shutdown()
        print('[Servidor 9] Servidor Cerrado')

### Ejecutar programas

Se deben ejecutar los programas en ambas máquinas para poder comunicarse entre sí.

In [ ]:
# Ejecutar primero el comando en máquina servidor:
python3 socket_servidor.py

# Ejecutar comando en máquina cliente:
python3 -u socket_cliente_practica_2.py

### Modificamos la api para poder jugar al Stop:

In [ ]:
from bottle import Bottle, run, response, request
import datetime
import subprocess
import string
import random
import socket

app = Bottle()

@app.route('/hi')
def hello():
    now = datetime.datetime.now()
    return f'Hola, hoy es {now.strftime("%d/%m/%Y")} y son las {now.strftime("%H:%M:%S")}'

@app.route('/status')
def status():
    process = subprocess.Popen(['systemctl', 'list-units', '--type=service', '--state=running'], stdout=subprocess.PIPE)
    output, error = process.communicate()
    lines = output.decode('utf-8').split('\n')
    services = []
    for line in lines:
        parts = line.split()
        if len(parts) >= 4:
            services.append({
                'unit': parts[0],
                'load': parts[1],
                'active': parts[2],
                'sub': parts[3],
                'description': ' '.join(parts[4:])
            })

    # Genera una tabla HTML
    html = '<table border="1">'
    html += '<tr><th>Unit</th><th>Load</th><th>Active</th><th>Sub</th><th>Description</th></tr>'
    for service in services:
        html += f'<tr><td>{service["unit"]}</td><td>{service["load"]}</td><td>{service["active"]}</td><td>{service["sub"]}</td><td>{service["description"]}</td></tr>'
    html += '</table>'

    response.content_type = 'text/html'
    return html


@app.route('/stop', method='POST')
def stop():
    # Código para detener el cliente
    HOST = "13.53.94.147"
    PORT = 50000

    # Crear un socket y conectar al servidor
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as mi_socket:
        try:
            mi_socket.connect((HOST, PORT))
            # Envía el comando 'cerrar' al servidor
            mi_socket.sendall(b'cerrar')
        except Exception as e:
            return f"Error al detener el cliente: {e}"
    
    return "Cliente detenido con éxito"

if __name__ == '__main__':
    run(app, host='0.0.0.0', port=8080)